<a href="https://colab.research.google.com/github/RejaulBSSE1324/ML/blob/main/1324_logistic_regression_lab_exercises.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚯 Lecture 11 Lab: Logistic regression and spam detection

<img src="https://github.com/joshuagrossman/mse125-labs-public/blob/main/hw5/img/spam-email.png?raw=1" alt= “spam-email” width="500" />

## ✅ Setup and data import
In this lab, we will work with a [classic dataset](https://archive.ics.uci.edu/dataset/94/spambase) of 4,601 emails classified as spam or not spam.

In [13]:
# Load in additional functions
# library(tidyverse)
# library(lubridate)

# Use three digits past the decimal point,
# and don't use scientific notation.
# options(digits = 3, scipen = 999)

# Format plots with a white background and dark features.
# theme_set(theme_bw())

# Increase the default text size of plots.
# If you are *not* working in Google Colab, we recommend commenting
# out this line of code.
# theme_update(text = element_text(size = 20))

# Increase the default plot width and height.
# If you are *not* working in Google Colab, we recommend commenting
# out this line of code.
# options(repr.plot.width=12, repr.plot.height=8)

# Read in the data
# spam = read_csv('https://jdgrossman.com/assets/spam.csv')

# peek at 10 random rows
# sample_n(spam, 10)

In [14]:
# Load libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Display settings
pd.set_option("display.precision", 3)      # 3 digits after decimal
pd.set_option("display.float_format", lambda x: f"{x:.3f}")  # no scientific notation

# Plot settings
sns.set_theme(style="whitegrid")           # White background with dark features
plt.rcParams['figure.figsize'] = (12, 8)   # Default plot width and height
plt.rcParams['font.size'] = 14             # Increase default text size

# Read in the data
spam = pd.read_csv("https://jdgrossman.com/assets/spam.csv")

# Peek at 10 random rows
spam.sample(10)


,make,address,all,3d,our,over,remove,internet,order,mail,...,char_semicolon,char_left_paren,char_left_bracket,char_exclamation,char_dollar,char_pound,capital_run_length_average,capital_run_length_longest,capital_run_length_total,is_spam
3939,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,1.860,...,0.189,0.757,0.000,0.000,0.000,0.000,2.630,16,171,0
4066,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,1.724,0.000,0.000,1.000,1,6,0
1135,0.000,0.000,0.000,0.000,0.360,0.360,0.000,0.360,0.360,0.360,...,0.000,0.000,0.000,0.653,0.118,0.000,1.530,13,75,1
951,0.000,0.000,0.320,0.000,0.640,0.640,0.320,0.320,0.000,0.000,...,0.000,0.111,0.000,0.000,0.166,0.000,1.494,10,139,1
2444,0.480,0.000,0.000,0.000,0.480,0.000,0.000,0.000,0.000,0.000,...,0.000,0.085,0.000,0.000,0.000,0.000,1.275,3,37,0
138,0.000,0.000,0.000,0.000,1.490,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,1.100,2,11,1
566,2.430,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.000,0.000,0.000,3.666,13,44,1
36,0.000,0.000,0.000,0.000,2.940,0.000,0.000,0.000,0.000,0.000,...,0.436,0.436,0.000,0.873,0.000,0.000,4.142,12,29,1
3375,0.110,0.000,0.110,0.000,0.000,0.000,0.000,0.000,1.150,0.000,...,0.047,0.159,0.031,0.000,0.031,0.000,3.196,32,505,0
1686,0.000,0.000,2.150,0.000,1.070,0.000,0.000,0.000,0.000,0.000,...,0.000,0.000,0.000,0.192,0.000,0.000,2.333,19,49,1


## ♨️ Warm up

How many emails are in the database?

What fraction of the emails in the database are spam?

Which email contains the highest percentage of words matching "money"? What percentage of words in that email match "money"?

In [15]:
# 1. How many emails are in the database?
num_emails = len(spam)
print("Number of emails:", num_emails)

# 2. What fraction of the emails are spam?
fraction_spam = spam['is_spam'].mean()   # since is_spam is 0/1
print("Fraction of emails that are spam:", fraction_spam)

# 3. Which email contains the highest percentage of words matching 'money'?
#    And what percentage is that?

max_money_row = spam.loc[spam['money'].idxmax()]
max_money_value = max_money_row['money']
max_money_index = spam['money'].idxmax()

print("Row index with highest 'money' percentage:", max_money_index)
print("Highest percentage of words matching 'money':", max_money_value)


Number of emails: 4601
Fraction of emails that are spam: 0.39404477287546186
Row index with highest 'money' percentage: 545
Highest percentage of words matching 'money': 12.5


## 🎲 Linear probability models (LPMs)

Fit a linear regression model to the spam data with the `lm` function.

Use the following covariates to predict the likelihood that an email is spam:
- `char_dollar`
- `credit`
- `money`
- `re`

How would you interpret the model coefficients for the intercept and for `char_dollar`?

- Note: `char_dollar` represents the percentage of characters in the email that match `$`.

In [16]:
import statsmodels.api as sm

# Select predictors
X = spam[['char_dollar', 'credit', 'money', 're']]

# Add intercept
X = sm.add_constant(X)

# Response variable
y = spam['is_spam']

# Fit linear regression (LPM)
lpm = sm.OLS(y, X).fit()

print(lpm.summary())


                            OLS Regression Results                            
Dep. Variable:                is_spam   R-squared:                       0.179
Model:                            OLS   Adj. R-squared:                  0.178
Method:                 Least Squares   F-statistic:                     250.3
Date:                Tue, 18 Nov 2025   Prob (F-statistic):          8.51e-195
Time:                        05:52:39   Log-Likelihood:                -2780.3
No. Observations:                4601   AIC:                             5571.
Df Residuals:                    4596   BIC:                             5603.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
const           0.3346      0.007     45.696      

Using your linear probability model and the `predict` function, predict the in-sample probability that each email is spam.

What is the smallest predicted probability? The largest? Do you notice any issues with these predictions?

In [17]:
# Predicted probabilities from the LPM
pred_prob = lpm.predict(X)

# Display first few predictions
pred_prob.head()

smallest = pred_prob.min()
largest = pred_prob.max()

print("Smallest predicted probability:", smallest)
print("Largest predicted probability:", largest)



Smallest predicted probability: -0.8125403869560923
Largest predicted probability: 3.8494089964108236


## 🎰 Odds functions

Write two functions:
- A function to convert probabilities to odds.
- A function to convert odds to probabilities

Test your functions by making sure that 2:1 odds returns a 2/3 probability, and vice versa.

Finally, suppose my probability of winning is 60%. If I double my odds of winning, what is my new probability of winning?

In [18]:
def prob_to_odds(p):
    return p / (1 - p)

def odds_to_prob(o):
    return o / (1 + o)

# Test the functions
# For 2:1 odds, probability should be 2/3
# For 2/3 probability, odds should be 2:1
p_test = 2/3
odds_test = prob_to_odds(p_test)
print(f"Probability {p_test:.3f} converts to odds {odds_test:.3f}")

odds_test_val = 2
p_test_val = odds_to_prob(odds_test_val)
print(f"Odds {odds_test_val:.3f} converts to probability {p_test_val:.3f}")


initial_p = 0.60
initial_odds = prob_to_odds(initial_p)
new_odds = 2 * initial_odds
new_p = odds_to_prob(new_odds)

print("New probability after doubling the odds =", new_p)

Probability 0.667 converts to odds 2.000
Odds 2.000 converts to probability 0.667
New probability after doubling the odds = 0.75


## 🪙 Fitting a logistic regression model

We can fit a logistic regression model with the same covariates as above with the following code:

In [19]:
import statsmodels.formula.api as smf

# Fit logistic regression model
model = smf.glm(formula='is_spam ~ char_dollar + credit + money + re', family=sm.families.Binomial(), data=spam).fit()

print(model.summary())

                 Generalized Linear Model Regression Results                  
Dep. Variable:                is_spam   No. Observations:                 4601
Model:                            GLM   Df Residuals:                     4596
Model Family:                Binomial   Df Model:                            4
Link Function:                  Logit   Scale:                          1.0000
Method:                          IRLS   Log-Likelihood:                -2213.9
Date:                Tue, 18 Nov 2025   Deviance:                       4427.8
Time:                        05:52:39   Pearson chi2:                 1.00e+10
No. Iterations:                     8   Pseudo R-squ. (CS):             0.3152
Covariance Type:            nonrobust                                         
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -1.0666      0.043    -24.680      

Interpret the intercept and `money` coefficients for the logistic regression model three different ways:
1. On the log odds scale
2. On the odds scale (by exponentiating the coefficients)
3. On the probability scale (using either the odds functions you wrote, or the divide by 4 trick).

Tip: Use the `coef` function to extract coefficients from the model.

In [20]:
# Extract coefficients
intercept_coef = model.params['Intercept']
money_coef = model.params['money']

print("--- Interpretations ---")

# 1. On the log odds scale
print("\n1. On the log odds scale:")
print(f"   Intercept (log odds): {intercept_coef:.3f}")
print(f"   Money (log odds): {money_coef:.3f}")
print("   - Interpretation of Intercept: When all other predictors are zero, the log odds of an email being spam is {intercept_coef:.3f}.")
print("   - Interpretation of Money: For a one-unit increase in the percentage of words matching 'money', the log odds of an email being spam increase by {money_coef:.3f}, holding other predictors constant.")

# 2. On the odds scale (by exponentiating the coefficients)
intercept_odds_ratio = np.exp(intercept_coef)
money_odds_ratio = np.exp(money_coef)

print("\n2. On the odds scale (Odds Ratios):")
print(f"   Intercept (odds ratio): {intercept_odds_ratio:.3f}")
print(f"   Money (odds ratio): {money_odds_ratio:.3f}")
print("   - Interpretation of Intercept: When all other predictors are zero, the odds of an email being spam are {intercept_odds_ratio:.3f} to 1.")
print("   - Interpretation of Money: For a one-unit increase in the percentage of words matching 'money', the odds of an email being spam are multiplied by {money_odds_ratio:.3f}, holding other predictors constant.")

# 3. On the probability scale
# Using the divide by 4 trick for a rough approximation near p=0.5
print("\n3. On the probability scale (approximate):")
# Approximate marginal effect for money (coefficient / 4)
money_prob_effect = money_coef / 4
print(f"   Approximate marginal effect for Money: A one-unit increase in 'money' is associated with an approximate {money_prob_effect:.3f} increase in the probability of being spam (near p=0.5).")


--- Interpretations ---

1. On the log odds scale:
   Intercept (log odds): -1.067
   Money (log odds): 1.993
   - Interpretation of Intercept: When all other predictors are zero, the log odds of an email being spam is {intercept_coef:.3f}.
   - Interpretation of Money: For a one-unit increase in the percentage of words matching 'money', the log odds of an email being spam increase by {money_coef:.3f}, holding other predictors constant.

2. On the odds scale (Odds Ratios):
   Intercept (odds ratio): 0.344
   Money (odds ratio): 7.340
   - Interpretation of Intercept: When all other predictors are zero, the odds of an email being spam are {intercept_odds_ratio:.3f} to 1.
   - Interpretation of Money: For a one-unit increase in the percentage of words matching 'money', the odds of an email being spam are multiplied by {money_odds_ratio:.3f}, holding other predictors constant.

3. On the probability scale (approximate):
   Approximate marginal effect for Money: A one-unit increase in 'mon